Install chencrafts:
`pip install chencrafts`

In [6]:
import numpy as np
import scqubits as scq
import matplotlib.pyplot as plt
import qutip as qt
import chencrafts as cc

In [7]:
exp_params = {
    'E_Ja': 5.645,
    'E_Jb': 4.85,
    'E_Ca': 0.952,
    'E_Cb': 0.9,
    'E_J': 4.246,
    'E_C': 12,
    'ECm': 8,
    'E_L1': 3.52,
    'E_L2': 3.52,
    # 'E_La': 0.292,
    # 'E_Lb': 0.287,
    'E_La': 0.29,
    'E_Lb': 0.29,

    'flux_s': 0.5 - 0.4483319171817633,
    'flux_c': 0.31710517597502225,
}

para = exp_params
para["flux_a"] = 0.5 + para["flux_s"]
para["flux_b"] = 0.5 - para["flux_s"]

In [8]:
sim_params = {
    "qa_cutoff": 110,
    "qb_cutoff": 110,
    "p_cutoff": 110,
    "m_cutoff": 110,
    "qa_dim": 5,
    "qb_dim": 5,
    "p_dim": 5,
    "m_dim": 7,
}

In [9]:
circ_yaml = f"""
branches:
# qubit a
- ["JJ", 0, 1, E_Ja = {para["E_Ja"]}, E_Ca = {para["E_Ca"]}]
- ["L", 1, 2, E_La = {para["E_La"]}]

# coupler
- ["C", 0, 2, E_C = {para["E_C"]}]
- ["L", 0, 2, E_L1 = {para["E_L1"]}]
- ["C", 0, 3, E_C]
- ["L", 0, 3, E_L2 = {para["E_L2"]}]
- ["JJ", 2, 3, E_J = {para["E_J"]}, ECm = {para["ECm"]}]

# qubit b
- ["JJ", 0, 4, E_Jb = {para["E_Jb"]}, E_Cb = {para["E_Cb"]}]
- ["L", 3, 4, E_Lb = {para["E_Lb"]}]
"""

ftc = scq.Circuit(circ_yaml, from_file=False)

trans_mat = np.linalg.inv([
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 1, -1, 0],
    [0, 1, 1, 0],
])

ftc.cutoff_ext_1 = sim_params["qa_cutoff"]
ftc.cutoff_ext_2 = sim_params["qb_cutoff"]
ftc.cutoff_ext_3 = sim_params["m_cutoff"]
ftc.cutoff_ext_4 = sim_params["p_cutoff"]

brch = ftc.branches
ftc.configure(
    transformation_matrix=trans_mat, 
    # system_hierarchy=[[1], [2], [3], [4]], 
    system_hierarchy=[[1], [2], [[3], [4]]], 
    subsystem_trunc_dims=[
        sim_params["qa_dim"], 
        sim_params["qb_dim"],
        [sim_params["m_dim"] * 2, [
            sim_params["m_dim"], 
            sim_params["p_dim"]
        ]], 
        # sim_params["m_dim"], 
        # sim_params["p_dim"],
    ],
    closure_branches=[
        brch[0], brch[7], brch[6]   # flux in junctions
    ],
    # closure_branches=[circ.branches[1], circ.branches[4]]   # flux in inductors
)

ftc.Φ1 = -para["flux_a"]
ftc.Φ2 = -para["flux_b"]
ftc.Φ3 = para["flux_c"]

ftc.sym_hamiltonian()

<IPython.core.display.Latex object>

In [10]:
evals = ftc.eigenvals(4)
evals - evals[0]

array([0.      , 0.048191, 0.062287, 0.110434])

## Sweet spot identification

In [4]:
from typing import Tuple

def find_sweetspot_by_spectrum(
    self,   # the ftc circuit
    flux_c = None, 
    evals_count=4,
    ftol = 1e-8,
    gtol = 1e-8,
    eps = 1e-11,
    mode = "l3",
    flux_s_bounds = [0, 0.1],
    flux_sb_bounds = None,      # if not None, flux_a and b are independent
    flux_c_bounds = [0.25, 0.3],
    run_num = 1,
    **kwargs
) -> Tuple[float, float, float]:
    
    def freq_ab(params, evals_count, mode):
        try:
            self.flux_a = 0.5 + params["flux_s"]
            self.flux_c = params["flux_c"]
            if flux_sb_bounds is None:
                self.flux_b = 0.5 - params["flux_s"]
            else:
                self.flux_b = 0.5 + params["flux_sb"]
        except Exception:
            self.Φ1 = 0.5 + params["flux_s"]
            if flux_sb_bounds is None:
                self.Φ2 = 0.5 - params["flux_s"]
            else:
                self.Φ2 = 0.5 + params["flux_sb"]
            self.Φ3 = params["flux_c"]

        eigs = self.eigenvals(evals_count)
        if mode == "l3":
            return eigs[3] - eigs[0]
        elif mode == "l1+l2":
            return eigs[1] + eigs[2] - 2 * eigs[0]
        elif mode == "l2-l1":
            return np.abs(eigs[2] - eigs[1])
        elif mode == "zz":
            abszz = np.abs(eigs[3] - eigs[2] - eigs[1] + eigs[0])
            return abszz
        elif mode == "l1+l2+zz":
            abszz = np.abs(eigs[3] - eigs[2] - eigs[1] + eigs[0])
            return eigs[1] + eigs[2] - 2 * eigs[0] + abszz
        else:
            raise ValueError("mode not supported")
        
    free_params = {"flux_s": flux_s_bounds, "flux_c": flux_c_bounds}
    if flux_sb_bounds is not None:
        free_params["flux_sb"] = flux_sb_bounds
    
    opt = cc.tb.Optimization(
        {}, free_params,
        freq_ab,
        optimizer="L-BFGS-B",
        target_kwargs={"evals_count": evals_count, "mode": mode},
        opt_options={"ftol": ftol, "gtol": gtol, "disp": False, "eps": eps, "maxls": 100}
    )

    if flux_c is not None:
        opt.fix(flux_c=flux_c)
    
    multi_opt = cc.tb.MultiOpt(opt)
    multi_traj = multi_opt.run(run_num)
    traj = multi_traj.best_traj()

    try:
        self.flux_a = 0.5 + traj.final_full_para["flux_s"]
        self.flux_c = traj.final_full_para["flux_c"]
        if flux_sb_bounds is None:
            self.flux_b = 0.5 - traj.final_full_para["flux_s"]
        else:
            self.flux_b = 0.5 + traj.final_full_para["flux_sb"]
        
        return self.flux_a, self.flux_b, self.flux_c
    except Exception:
        self.Φ1 = 0.5 + traj.final_full_para["flux_s"]
        if flux_sb_bounds is None:
            self.Φ2 = 0.5 - traj.final_full_para["flux_s"]
        else:
            self.Φ2 = 0.5 + traj.final_full_para["flux_sb"]
        self.Φ3 = traj.final_full_para["flux_c"]
        
        return self.Φ1, self.Φ2, self.Φ3


In [11]:
phi1, phi2, phi3 = find_sweetspot_by_spectrum(
    ftc, 
    # flux_c = 0.1,
    flux_s_bounds = [-0.1, 0.1],
    flux_c_bounds = [0.0, 0.5],
    run_num=10,
)
# print(phi1, phi2, phi3)

 /Users/apple/PycharmProjects/scqubits/scqubits/core/circuit_routines.py: 2700

KeyboardInterrupt: 

In [12]:
evals = ftc.eigenvals(4)
evals - evals[0]


array([0.      , 0.041009, 0.073856, 0.114413])

## Drive & eff coupling

In [13]:
hspace = ftc.hilbert_space
dims = hspace.subsystem_dims
hamiltonian = qt.Qobj(ftc.hamiltonian(), dims=[dims, dims])

theta1_op = qt.Qobj(ftc.θ1_operator(), dims=[dims, dims])
theta2_op = qt.Qobj(ftc.θ2_operator(), dims=[dims, dims])
theta3_op = qt.Qobj(ftc.θ3_operator(), dims=[dims, dims])
drive_op = (
    (ftc.E_La + ftc.E_Lb) / 2 * (theta1_op - theta2_op) / 2
    - (ftc.E_L1 + ftc.E_L2 + ftc.E_La + ftc.E_Lb) / 4 * theta3_op
)

# Q2_op = qt.Qobj(ftc.Q2_operator(), dims=[dims, dims])
# drive_op = Q2_op

In [20]:
theta1_op

Quantum object: dims=[[5, 5, 14], [5, 5, 14]], shape=(350, 350), type='oper', dtype=CSR, isherm=True
Qobj data =
[[-2.611284  0.        0.       ...  0.        0.        0.      ]
 [ 0.       -2.611284  0.       ...  0.        0.        0.      ]
 [ 0.        0.       -2.611284 ...  0.        0.        0.      ]
 ...
 [ 0.        0.        0.       ... -0.508956  0.        0.      ]
 [ 0.        0.        0.       ...  0.       -0.508956  0.      ]
 [ 0.        0.        0.       ...  0.        0.       -0.508956]]

In [14]:
flux_shift = 0.1

evals, evecs = hamiltonian.eigenstates(hspace.dimension, tol=1e-10)

subspace_state_idx = [0, 1, 2, 3]
subspace1_evecs = [evecs[idx] for idx in subspace_state_idx]
subspace2_evecs = [evec for idx, evec in enumerate(evecs) if idx not in subspace_state_idx]

H_sw, _, _ = cc.cqed.block_diagonalize_pymablock(
    [hamiltonian, drive_op * flux_shift * np.pi * 2],     
    # flux drive amp
    subspace_eigenvectors=[subspace1_evecs, subspace2_evecs],
    atol=1e-10,
)

 /Users/apple/miniconda3/envs/flux/lib/python3.12/site-packages/pymablock/block_diagonalization.py: 1091

In [19]:
drive_op

Quantum object: dims=[[5, 5, 14], [5, 5, 14]], shape=(350, 350), type='oper', dtype=CSR, isherm=True
Qobj data =
[[-1.901168e+00  3.048116e+00 -6.511733e-01 ...  0.000000e+00  0.000000e+00  0.000000e+00]
 [ 3.048116e+00  1.002994e+00  4.086250e+00 ...  0.000000e+00  0.000000e+00  0.000000e+00]
 [-6.511733e-01  4.086250e+00 -2.794648e-01 ...  0.000000e+00  0.000000e+00  0.000000e+00]
 ...
 [ 0.000000e+00  0.000000e+00  0.000000e+00 ... -1.313851e+00 -4.385411e-13  3.563676e-12]
 [ 0.000000e+00  0.000000e+00  0.000000e+00 ... -4.385411e-13  3.651534e-01  8.060460e-14]
 [ 0.000000e+00  0.000000e+00  0.000000e+00 ...  3.563676e-12  8.060460e-14 -1.256549e+00]]

In [15]:
H_sw[0, 0, 0]

array([[ 1.231803e+01+4.700016e-16j, -3.926052e-13+4.600174e-13j,  3.628972e-13+1.043392e-13j, -6.769908e-13+4.808773e-14j],
       [-3.865996e-13-4.592886e-13j,  1.235904e+01+2.359549e-15j, -4.152916e-13-1.481080e-12j,  4.866723e-14-1.701380e-13j],
       [ 3.604209e-13-1.053665e-13j, -4.142938e-13+1.480925e-12j,  1.239188e+01+7.749877e-16j,  1.267992e-13-5.634490e-13j],
       [-6.762773e-13-5.271499e-14j,  5.028248e-14+1.730025e-13j,  1.283771e-13+5.617000e-13j,  1.243244e+01-2.017483e-15j]])

In [16]:
H_sw[0, 0, 1]

array([[-0.654984+7.141165e-17j,  0.069927-2.875929e-02j,  0.017262+1.640791e-03j, -0.014874+4.612415e-02j],
       [ 0.069927+2.875929e-02j, -0.64724 +4.301572e-17j,  0.017194+9.060010e-03j, -0.030216+3.571841e-02j],
       [ 0.017262-1.640791e-03j,  0.017194-9.060010e-03j, -0.725805-5.196717e-17j, -0.002442+1.106788e-02j],
       [-0.014874-4.612415e-02j, -0.030216-3.571841e-02j, -0.002442-1.106788e-02j, -0.698891-7.874018e-18j]])

In [17]:
H_sw[0, 0, 2]

array([[-0.469558+0.j      , -0.097794+0.04022j , -0.023071-0.002193j,  0.004898-0.015188j],
       [-0.097794-0.04022j , -0.444271+0.j      , -0.003927-0.002069j,  0.050597-0.05981j ],
       [-0.023071+0.002193j, -0.003927+0.002069j, -0.432491+0.j      ,  0.003905-0.017696j],
       [ 0.004898+0.015188j,  0.050597+0.05981j ,  0.003905+0.017696j, -0.433482+0.j      ]])

In [18]:
H_sw[0, 0, 3]

array([[ 0.065716-8.552915e-18j,  0.015565-6.401320e-03j,  0.003568+3.391723e-04j,  0.000552-1.710527e-03j],
       [ 0.015565+6.401320e-03j,  0.058758+6.881349e-18j, -0.001054-5.556137e-04j, -0.008768+1.036407e-02j],
       [ 0.003568-3.391723e-04j, -0.001054+5.556137e-04j,  0.065174-9.546061e-18j, -0.00067 +3.037315e-03j],
       [ 0.000552+1.710527e-03j, -0.008768-1.036407e-02j, -0.00067 -3.037315e-03j,  0.062345-1.065695e-17j]])

In [8]:
H_sw[0, 0, 4]

NameError: name 'H_sw' is not defined

In [ ]:
H_sw[0, 0, 5]

array([[-0.000335+8.819492e-19j,  0.000241+1.307243e-03j,  0.00112 +8.592790e-05j,  0.001016-4.974699e-04j],
       [ 0.000241-1.307243e-03j, -0.000527-2.024955e-18j, -0.000292+1.102070e-03j,  0.000352+1.255994e-03j],
       [ 0.00112 -8.592790e-05j, -0.000292-1.102070e-03j, -0.000154-2.424863e-18j, -0.000963+5.670819e-04j],
       [ 0.001016+4.974699e-04j,  0.000352-1.255994e-03j, -0.000963-5.670819e-04j, -0.000297-2.299607e-18j]])